In [ ]:
!pip install google-cloud-storage rasterio numpy pillow tqdm --quiet
!pip install torch torchvision scikit-image --quiet

In [ ]:
# Download Files from GCP
import os
from google.cloud import storage
import zipfile
from tqdm import tqdm
from google.colab import auth

# Step 1: Re-authenticate with Google Cloud
print("Authenticating with Google Cloud...")
auth.authenticate_user()

# Step 2: Initialize GCS client
try:
    client = storage.Client(project='YOUR_GCP_PROJECT_NAME')
    bucket_name = 'YOUR_GCP_BUCKET_NAME'
    bucket = client.get_bucket(bucket_name)
    print(f"Successfully connected to bucket: {bucket_name}")
except Exception as e:
    print(f"Error connecting to GCS: {e}")
    raise

# Step 3: Define paths
gcs_zip_path = 'SR/compressed/compressed.zip'  # Path to zip file in GCS
local_zip_path = '/content/compressed.zip'  # Local path for downloaded zip
extract_folder = '/content/extracted_data'  # Folder to extract the zip contents

# Step 4: Download zip file from GCS
try:
    blob = bucket.blob(gcs_zip_path)
    os.makedirs(os.path.dirname(local_zip_path), exist_ok=True)
    blob.download_to_filename(local_zip_path)
    print(f"Downloaded gs://{bucket_name}/{gcs_zip_path} to {local_zip_path}")
except Exception as e:
    print(f"Error downloading zip file: {e}")
    raise

# Step 5: Extract zip file
try:
    os.makedirs(extract_folder, exist_ok=True)
    with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
        zip_files = zip_ref.namelist()
        for file in tqdm(zip_files, desc="Extracting files"):
            zip_ref.extract(file, extract_folder)
    print(f"Extracted zip file to {extract_folder}")
except Exception as e:
    print(f"Error extracting zip file: {e}")
    raise

# Step 6: Clean up local zip file
try:
    os.remove(local_zip_path)
    print("Removed local zip file")
except Exception as e:
    print(f"Error removing local zip file: {e}")

In [ ]:
import os
import glob
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision.models import vgg19
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import train_test_split
from skimage.metrics import peak_signal_noise_ratio, structural_similarity
from tqdm import tqdm
import torchvision.transforms as transforms
import rasterio
import numpy as np
import matplotlib.pyplot as plt
import logging

# Logging setup
logging.basicConfig(level=logging.INFO, format="%(message)s")
logger = logging.getLogger()

# Check device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

In [ ]:
# For Model Acehitecture Please Drop an email muddasirshah@outlook.com